In [ ]:
%load_ext autoreload
%autoreload 2

from humbldata.toolbox.toolbox_controller import Toolbox

toolbox = Toolbox(
    symbol="AAPl,msft",
    interval="1d",
    start_date="2021-01-01",
    end_date="2021-01-01",
)



In [ ]:
toolbox

In [ ]:
result = toolbox.technical.mandelbrot_channel(window="1m")

In [ ]:
result

In [31]:
from humbldata.core.standard_models.toolbox.technical.mandelbrotchannel import (
    MandelbrotChannelQueryParams,
)


MandelbrotChannelQueryParams(window="60 days", rs_method="RS")



MandelbrotChannelQueryParams(window=60d, rv_adjustment=True, rv_method=std, rs_method=RS, rv_grouped_mean=False, live_price=False)

## Test validation

In [ ]:
from openbb import obb

context_params = toolbox

equity_historical_data = (
    obb.equity.price.historical(
        symbol="AAPL,MSFT",
        start_date=context_params.start_date,
        end_date=context_params.end_date,
        provider=context_params.provider,
    ).to_polars()
).drop(["dividends", "stock_splits"])


In [ ]:
print(equity_historical_data.dtypes)
print(equity_historical_data.columns)


In [ ]:
%load_ext autoreload
%autoreload 2

from humbldata.core.standard_models.toolbox import ToolboxData


ToolboxData(equity_historical_data)



In [35]:
import polars as pl
import datetime

from humbldata.core.standard_models.toolbox.technical.mandelbrotchannel import MandelbrotChannelData

test_frame = pl.LazyFrame(
    {
        "date": pl.date_range(
            start=datetime.date(2021, 1, 1),
            end=datetime.date(2021, 1, 3),
            eager=True,
        ),
        "symbol": ["AAPL", "AAPL", "AAPL"],
        "bottom_price": [100.0, 100.0, 100.0],
        "recent_price": [110.0, 110.0, 110.0],
        "top_price": [120.0, 120.0, 120.0],
    }
)

MandelbrotChannelData(test_frame).collect()



date,symbol,bottom_price,recent_price,top_price
date,str,f64,f64,f64
2021-01-01,"""AAPL""",100.0,110.0,120.0
2021-01-02,"""AAPL""",100.0,110.0,120.0
2021-01-03,"""AAPL""",100.0,110.0,120.0


## Reprex

In [ ]:
from typing import Optional
import pandera.polars as pa
import polars as pl


class ModelWithChecks(pa.DataFrameModel):
    a: int
    b: str
    c: Optional[float]
    date: pl.Date


valid_lf = pl.LazyFrame(
    {
        "a": pl.Series([1, 2, 3], dtype=pl.Int64),
        "b": ["d", "e", "f"],
        # "c": [0.0, 0.6, 1],
        "date": pl.date_range(
            start=datetime.date(2021, 1, 1),
            end=datetime.date(2021, 1, 3),
            eager=True,
        ),
    }
)


ModelWithChecks.validate(valid_lf, lazy=True).collect()
